## distilBERT implentation of Sentiment analysis using DivSigma's Dataset

In [1]:
import numpy as np
import pandas as pd
import torch
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
# !rm contestant_judgment.csv 
!wget -nc https://raw.githubusercontent.com/Codeseys/IgnitionHacks2020DivSigma/master/contestant_judgment.csv 
# !rm training_data.csv 
!wget -nc https://raw.githubusercontent.com/Codeseys/IgnitionHacks2020DivSigma/master/training_data.csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codeseys/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codeseys/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/codeseys/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/codeseys/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
--2020-08-31 18:38:57--  https://raw.githubusercontent.com/Codeseys/IgnitionHacks2020DivSigma/master/contestant_judgment.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)...151.101.128.133, 151.101.64.133, 151.101.0.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.128.133|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 56472148 (54M) [

In [2]:
df = pd.read_csv('training_data.csv')
df = df.drop(['ID','User'],axis=1)
df.head()

,Text,Sentiment
0,I *heart* filling up @dennisschaub desk 1 it...,1
1,"#SocioMat - people create prettier, younger an...",1
2,no way i dont want the tour to end,0
3,@HemalRadia Hi Amazing Brother! Sending Limitl...,1
4,@flockmaster they are chocolate,1


In [3]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [4]:
stopwordslist =  set(stopwords.words('english'))
lemmatty = WordNetLemmatizer()
wordnetmap = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def removestopwords(text):    
    return " ".join([word for word in str(text).split() if word not in stopwordslist])
def lemmy(text):
    pos_text = nltk.pos_tag(str(text).split())
    return " ".join([lemmatty.lemmatize(word,wordnetmap.get(pos[0],wordnet.NOUN))for word, pos in pos_text])    

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text = re.sub(r'#','',text)
    
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    
    text = lemmy(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]','', text)
    text = removestopwords(text)
    
    return text

In [5]:
df['Text']=df['Text'].apply(clean_text)

In [6]:
tokenized = df['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [7]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_masked_ = np.where(padded!=0,1,0)

In [8]:
input_ids = torch.tensor(np.array(padded))
attention_mask = torch.tensor(attention_masked_)

with torch.no_grad():
    last_hidden_states = model(input_ids,attention_mask=attention_mask)

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 390144000000 bytes. Error code 12 (Cannot allocate memory)

In [9]:
features = last_hidden_states[0][:,0,:].numpy()

NameError: name 'last_hidden_states' is not defined

In [10]:
labels = df[1]
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

KeyError: 1

In [11]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

NameError: name 'train_features' is not defined

In [12]:
lr_clf.score(test_features, test_labels)

NameError: name 'test_features' is not defined